За ранее извиняюсь, с ячейками Markdown так еще и не научился работать. Но если открыть в VScode и нажать на нее. Высветится корректный и понятный код SQL, который я написал ниже. Прошу понят и простить.

In [1]:
import math 
from scipy.stats import norm
from scipy import stats as st
from scipy.stats import t
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import psycopg2
from itertools import combinations
from collections import Counter
from collections import defaultdict

В нашей базе данных, есть 2 таблицы с исходными данными:

Таблица carts — данные о пользовательских корзинах 
Promo Code ID — ID промокода, если он есть 
Purchased At — дата оплаты 
User ID — ID пользователя 
Created At — дата создания корзины 
Updated At — дата последнего обновления информации 
ID — идентификатор корзины 
State — состояние оплаты 

Таблица cart items — данные о курсах, которые пользователи добавили в корзину 
Created At — дата создания события 
Resource Type — тип продукта 
Resource ID — ID курса 
Cart ID — идентификатор корзины 
Updated At — дата последнего обновления информации 
ID — идентификатор операции


Посмотрим за какие года у нас есть информация о продажах

SELECT
    distinct(extract(year from purchased_at))
FROM final.cart_items AS items
    JOIN final.carts ON items.cart_id = carts.id
WHERE resource_type = 'Course' AND state = 'successful' 

На выходе видим, что это 2017 и 2018 года.

Далее хочу посмотреть, сколько же клиентов покупало у нас курсы:

SELECT
    count(distinct user_id)
FROM final.cart_items AS items
    JOIN final.carts ON items.cart_id = carts.id
WHERE resource_type = 'Course' AND state = 'successful' 

это 49006 человек.

Теперь интересно сколько же всего курсов на нашей платформе:

SELECT 
    count (distinct resource_id) as cntt
FROM final.cart_items as ci
    LEFT JOIN final.carts as c on ci.cart_id = c.id
WHERE resource_type = 'Course'

Итог: 127

Отлично, теперь поймем каково среднее число купленных курсов на одного клиента
Для этого посчитаем сколько приходится покупок для каждого клиента, создадим из этого подзапрос. И уже в финальном запросе, попросим среднее число.

WITH cnt as 
(SELECT 
    user_id,
    count (resource_id) as cntt
FROM final.cart_items as ci
    LEFT JOIN final.carts as c on ci.cart_id = c.id
WHERE resource_type = 'Course' and state = 'successful'
GROUP BY 1
)

SELECT
    avg(cntt)
FROM cnt

Итог: 1.44

Теперь поймем, сколько клиентов купило больше одного курса.

WITH cnt as 
(SELECT 
    user_id,
    count (resource_id) as cntt
FROM final.cart_items as ci
    LEFT JOIN final.carts as c on ci.cart_id = c.id
WHERE resource_type = 'Course' and state = 'successful'
GROUP BY 1
HAVING count(distinct resource_id)>1)

SELECT
    count(user_id)
FROM cnt

Итог: 12656

Получим необходимые нам данные, это пользователи, купившие более одного курса и ID эти самых курсов.

WITH cnt as 
(SELECT 
    user_id,
    count (resource_id) as cntt
FROM final.cart_items as ci
    LEFT JOIN final.carts as c on ci.cart_id = c.id
WHERE resource_type = 'Course' and state = 'successful'
GROUP BY 1
HAVING count(distinct resource_id)>1)

SELECT 
    cnt.user_id,
    resource_id
FROM cnt
    JOIN final.carts ON cnt.user_id = carts.user_id
    JOIN final.cart_items  AS items ON items.cart_id = carts.id
WHERE resource_type = 'Course' and state = 'successful'
ORDER BY 1 ASC

Выполним этот же запрос, но с помощью python. В итоге получаем DataFrame с точностью такой же таблицей, как и в sql запросе. 

In [2]:
def sql():
    # Пытаемся соединится с нашей БД и пытаемся предугадать ошибку.
    try:
        connection = psycopg2.connect(
            dbname='skillfactory',
            user='skillfactory', 
            host='84.201.134.129', 
            password='cCkxxLVrDE8EbvjueeMedPKt', 
            port=5432
        )
        cursor = connection.cursor()
        
        # делаем запрос sql который описывали выше
        query = '''WITH cnt as 
    (SELECT 
        user_id,
        count (resource_id) as cntt
    FROM final.cart_items as ci
        LEFT JOIN final.carts as c on ci.cart_id = c.id
    WHERE resource_type = 'Course' and state = 'successful'
    GROUP BY 1
    HAVING count(distinct resource_id)>1)

    SELECT 
        distinct(cnt.user_id),
        resource_id
    FROM cnt
        JOIN final.carts ON cnt.user_id = carts.user_id
        JOIN final.cart_items  AS items ON items.cart_id = carts.id
    WHERE resource_type = 'Course' and state = 'successful'
    ORDER BY 1 ASC 
        '''  
        
        cursor.execute(query)  
        request = cursor.fetchall()   
        connection.commit()
        return(pd.DataFrame(request, columns=('user_id', 'id_course'))) # если все хорошо. Возвращаем табоицу

    except (Exception, psycopg2.Error) as error: # пытаемся предугадать ошибку
        if connection:
            print("Error", error)
    finally:
        if connection:
            cursor.close()
            connection.close()  # в любом случае закрываем наш запрос
            
df = (sql())
df


,user_id,id_course
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125
...,...,...
34069,2188926,515
34070,2188926,743
34071,2190141,756
34072,2190141,794


Далее хотим разбить все курсы по парам, что бы можно было понять, какие пары наиболее эффективны в продажахю

In [3]:
# Создаем пустой словарь для результата
result_dict = {}

# Проходимся по строкам DataFrame
for index, row in df.iterrows():
    key = row['user_id']
    value = row['id_course']
    
    # Проверяем, есть ли ключ уже в словаре
    if key in result_dict:
        # Если ключ уже есть, то добавляем значение в список
        result_dict[key].append(value)
    else:
        # Если ключа нет, то создаем новую запись в словаре с ключом и значением в виде списка
        result_dict[key] = [value]

# Создаем список списков наших курсов
list_of_course = []
for i in result_dict.values(): 
    list_of_course.append(i) 

for lst in list_of_course: # Сортируем наши списки, что бы пары посчитались вместе
    lst.sort()

# Генерация комбинаций значений внутри внутренних списков
combs = []
for lst in list_of_course:
    lst_combs = list(combinations(lst, 2))
    combs.extend(lst_combs)

# Посчитаем с помощью Counter какие пары чаще всего встречаются
cnt = Counter()
for i in combs:
    cnt[i] += 1

cnt = dict(cnt)

pair_popular = pd.DataFrame({'Courses': cnt.keys(),
                             'Popular': cnt.values()})

# сгрупировали нашу таблицу, что иметь общие рузультаты
pair_popular = pair_popular.groupby('Courses')['Popular'].sum().sort_values(ascending=False).reset_index() 

# сделал таким образом, что бы можно было удобно работать в дальнейшем с таблицей пар
pair_popular 

,Courses,Popular
0,"(551, 566)",797
1,"(515, 551)",417
2,"(489, 551)",311
3,"(523, 551)",304
4,"(566, 794)",290
...,...,...
3984,"(753, 814)",1
3985,"(753, 810)",1
3986,"(753, 803)",1
3987,"(513, 750)",1


Далее найдем самые популярные курсы среди наших пользователей, для того, что бы заменить малопопулярный курс. Так как у нас не понятно что это за курсы, я не могу рекомендовать темы, которые могут быть интересны покупателю. Поэтому возьмем просто самый популярный. 

In [4]:
# создали таблицу с курсами, которые чаще всего покупались
most_popular_course = pd.DataFrame(df['id_course'].value_counts().reset_index()).rename(columns={"index": "Course", 
                                                                                                 "id_course": "Popularity"}) 

# Посмотрим, что чаще всего покупалось. 
most_popular_course 


,Course,Popularity
0,551,2935
1,566,2342
2,515,1311
3,489,1125
4,490,1084
...,...,...
121,833,3
122,911,3
123,1201,2
124,1199,2


Создадим таблицу с рекомендациями. 

In [7]:
recommendations_df = pd.DataFrame(columns=['Course', 'Recommendation1', 'Recommendation2','Recommendation3'])


for course in most_popular_course['Course']: # Курсы из таблицы популярности отдельных курсов     
    recommendation = defaultdict()
    
    for pair in pair_popular['Courses']: # курсы из таблицы пар
        
        # если курс есть в паре
        if course in pair: 
            
            # и если ключ из словаря существует, добовляем значение
            if course in recommendation: 
                recommendation[course].extend([c for c in pair if c != course])
                
            # если ключ отсутсвует, то добавляем его со значением 
            else: 
                recommendation[course] = [c for c in pair if c != course]
                
    # print(recommendation) # получили списки с ключом - курс и значения - все курсы когда либо покупавшие вместе с ним.
    
    # Создаем таблицу с курсами и рекомендациями к ним.
    for key, values in recommendation.items():
        recommendations_df = recommendations_df.append({'Course': key, 
                                                    'Recommendation1': values[0], 
                                                    'Recommendation2': values[1],
                                                    'Recommendation3': values[2]}, 
                                                   ignore_index=True) 

display(recommendations_df)
        

        

/var/folders/cn/scf3zm9942105gh6p3bt09_00000gn/T/ipykernel_39836/1891602660.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({'Course': key,
/var/folders/cn/scf3zm9942105gh6p3bt09_00000gn/T/ipykernel_39836/1891602660.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({'Course': key,
/var/folders/cn/scf3zm9942105gh6p3bt09_00000gn/T/ipykernel_39836/1891602660.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({'Course': key,
/var/folders/cn/scf3zm9942105gh6p3bt09_00000gn/T/ipykernel_39836/1891602660.py:24: FutureWarning: The frame.append method is deprecated and will be removed f

,Course,Recommendation1,Recommendation2,Recommendation3
0,551,566,515,489
1,566,551,794,490
2,515,551,489,523
3,489,551,515,523
4,490,566,551,809
...,...,...,...,...
121,833,507,570,552
122,911,514,803,908
123,1201,1125,1144,1186
124,1199,523,359,1115


Отлично, создали таблицу с рекомендациями курсов исходя из популярности покупок в парах. Для начала мы создали список с самыми популярными парами и отсортировали его. Далее Получили список с рекомендациями для каждого курса(соответсвенно: ключ - это курс, а значения - это все возможные варианты его пар, когда либо покупали пользователи). 

Так как у нас pair_popular уже была отсортирована, наш цикл добовлял последовательно все эллементы. В начале получили самые покупаемые варианты, а в самом конце - не популярные.

А далее в итоговую таблицу просто добавили значения по списку:  
            1 рекомендация - самая популярная в покупке наших пар.
            2 рекомендация - это вторая по популярноси пара для рассматриваемого курса
            3 рекомендация - соответсвенно 3 по счету в нашем списке.

Как мы уже знаем, не все курсы пользуются популярностью и некоторые пары имеют слабую связь. Думаю стоит определить минимальную границу для рекомендаций. Например. Если курсы покупались менее 200 раз, то возможно стоит предлагать популярные курсы, которые больше заинтересуют клиента.  

Выбрал такой вариант, потому, что у нас есть только ID и я не могу понять темы данного субъекта. 
Возможно если бы это были хотя бы названия курсов, то можно было бы использовать и логическую связь между ними, а потом уже популярность курсов.

Следуйщий шаг. Отфильтруем таблицу по условию, что курсы, которые покупали менее 200 раз, не будет предлагать пользователю, а заменим их на самые популярные. Первые два курса не стал включать в рекомендации. Они и так слишком часто попадаются в таблице. И скорее всего - это те курсы, по которым знают наш сайт. (Это который 'Python для чайников').

In [6]:
# Выберем не популярные и популярные курсы и добавим их в список
unpopular = most_popular_course[most_popular_course['Popularity'] < 200]
unpopular_courses = []

popular = most_popular_course[most_popular_course['Popularity'] > 1000]
popular_course = []

for course in unpopular['Course']:
    unpopular_courses.append(course)
    
for course in popular['Course']:
    popular_course.append(course)
    
# Замена значений в 'Recommendation1'
recommendations_df['Recommendation1'] = recommendations_df['Recommendation1'].replace(unpopular_courses, popular_course[2])

# Замена значений в 'Recommendation2'
recommendations_df['Recommendation2'] = recommendations_df['Recommendation2'].replace(unpopular_courses, popular_course[3])

# Замена значений в 'Recommendation3'
recommendations_df['Recommendation3'] = recommendations_df['Recommendation3'].replace(unpopular_courses, popular_course[4])

recommendations_df

,Course,Recommendation1,Recommendation2,Recommendation3
0,551,566,515,489
1,566,551,794,490
2,515,551,489,523
3,489,551,515,523
4,490,566,551,809
...,...,...,...,...
121,833,507,570,552
122,911,514,489,908
123,1201,1125,489,490
124,1199,523,489,1115


Таким образм мы получили таблицу с рекомендациями. Она хороша тем, что можно легко добавлять новые рекомендации или редактировать уже имеющие. Думаю, что код получился у меня динамичный. Просто добавить столбец в DF, а потом значение в нашем цикле. 
Свободно можно манипулировать минимальной границей популярности курса, что бы производить фильтрацию финальной таблицы.  

Как можно видеть, в первом варианте таблицы, без фильтров и готовом варианте. Верхние курсы, самые попоулярные и их рекомендации не отличаются. А вот нижние уже подверглись фильтру.